In [1]:
import sys
sys.path.append("../src")
import os

import pandas as pd
import numpy as np
from etf_transformations import *
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from scipy.stats import randint, uniform

In [2]:
df = pd.read_csv('../data/preprocessed/agg_emb/XLK_v4.csv')

df = sign_next_day(df)
df = drop_sign_and_return(df)

df.head()

,Date,Price,avg_positive_XLK,avg_neutral_XLK,avg_negative_XLK,n_XLK,sent_index_XLK,emb_0,emb_1,emb_2,...,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day
0,2018-03-20,63.310219,0.258843,0.683649,0.057507,6.0,0.201336,-0.040082,-0.042018,0.012110,...,-0.006043,-0.045790,-0.053740,0.008389,0.029547,-0.061633,0.056060,0.029845,1.0,-1.0
1,2018-03-21,62.929611,0.079694,0.891288,0.029018,3.0,0.050676,-0.055223,-0.005217,0.019322,...,0.039590,-0.018541,-0.059512,0.014814,0.063403,-0.086791,0.030065,0.011303,1.0,-1.0
2,2018-03-22,61.305096,0.348509,0.226298,0.425194,5.0,-0.076685,-0.029817,0.008005,0.017561,...,0.002415,0.003987,-0.066090,0.015980,0.000298,-0.078723,0.040454,0.003072,1.0,-1.0
3,2018-03-23,59.671288,0.241439,0.482122,0.276438,5.0,-0.034999,-0.015262,-0.058099,-0.005390,...,-0.027297,-0.014804,0.027459,-0.004768,0.018634,-0.081501,-0.006712,0.049573,1.0,1.0
4,2018-03-26,61.936344,0.234637,0.625601,0.139762,6.0,0.094875,-0.029753,-0.008020,0.026985,...,-0.016723,-0.013599,-0.027924,0.006294,0.021656,-0.090146,0.043457,0.037637,1.0,-1.0


In [3]:
df["Date"] = pd.to_datetime(df["Date"]).dt.normalize()
df = df.sort_values("Date").reset_index(drop=True)

df["Return_next_day"] = np.log(df["Price"].shift(-1) / df["Price"])
df["AbsReturn_next_day"] = df["Return_next_day"].abs()

df["no_news"] = df["n_XLK"].isna().astype(int) # a day where there were no sector specific news
df.head(10)


,Date,Price,avg_positive_XLK,avg_neutral_XLK,avg_negative_XLK,n_XLK,sent_index_XLK,emb_0,emb_1,emb_2,...,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day,Return_next_day,AbsReturn_next_day,no_news
0,2018-03-20,63.310219,0.258843,0.683649,0.057507,6.0,0.201336,-0.040082,-0.042018,0.012110,...,0.008389,0.029547,-0.061633,0.056060,0.029845,1.0,-1.0,-0.006030,0.006030,0
1,2018-03-21,62.929611,0.079694,0.891288,0.029018,3.0,0.050676,-0.055223,-0.005217,0.019322,...,0.014814,0.063403,-0.086791,0.030065,0.011303,1.0,-1.0,-0.026154,0.026154,0
2,2018-03-22,61.305096,0.348509,0.226298,0.425194,5.0,-0.076685,-0.029817,0.008005,0.017561,...,0.015980,0.000298,-0.078723,0.040454,0.003072,1.0,-1.0,-0.027012,0.027012,0
3,2018-03-23,59.671288,0.241439,0.482122,0.276438,5.0,-0.034999,-0.015262,-0.058099,-0.005390,...,-0.004768,0.018634,-0.081501,-0.006712,0.049573,1.0,1.0,0.037256,0.037256,0
4,2018-03-26,61.936344,0.234637,0.625601,0.139762,6.0,0.094875,-0.029753,-0.008020,0.026985,...,0.006294,0.021656,-0.090146,0.043457,0.037637,1.0,-1.0,-0.032600,0.032600,0
5,2018-03-27,59.949760,0.075072,0.531589,0.393340,7.0,-0.318268,-0.059012,-0.016827,0.007342,...,-0.004143,0.051210,-0.061163,0.044529,0.055805,1.0,-1.0,-0.006680,0.006680,0
6,2018-03-28,59.550602,0.310788,0.477132,0.212080,9.0,0.098708,-0.038110,-0.029604,0.028321,...,0.028669,-0.007364,-0.095580,-0.004168,0.057462,1.0,1.0,0.019604,0.019604,0
7,2018-03-29,60.729534,0.370141,0.613385,0.016474,3.0,0.353667,-0.021918,0.029772,0.047880,...,0.009561,0.067820,-0.044205,0.060755,0.026523,1.0,-1.0,-0.024134,0.024134,0
8,2018-04-02,59.281403,0.134719,0.351496,0.513785,10.0,-0.379066,-0.038686,-0.003821,0.047725,...,0.009345,0.023480,-0.090401,0.017328,0.019723,1.0,1.0,0.009817,0.009817,0
9,2018-04-03,59.866215,0.196441,0.353618,0.449942,9.0,-0.253501,-0.014222,-0.011880,0.029031,...,0.009188,0.032312,-0.081563,0.019184,0.024981,1.0,1.0,0.013859,0.013859,0


In [4]:
q = 0.70  # try 0.70, 0.75, 0.80
tau = df["AbsReturn_next_day"].quantile(q)
df["HighVol_next_day"] = (df["AbsReturn_next_day"] > tau).astype(int)


df["abs_ret_t"] = np.log(df["Price"] / df["Price"].shift(1)).abs()
df["vol_5"] = df["abs_ret_t"].rolling(5).mean()


df = df.dropna(subset=["HighVol_next_day"])
df.head(10)

,Date,Price,avg_positive_XLK,avg_neutral_XLK,avg_negative_XLK,n_XLK,sent_index_XLK,emb_0,emb_1,emb_2,...,emb_382,emb_383,is_trading_day,Sign_next_day,Return_next_day,AbsReturn_next_day,no_news,HighVol_next_day,abs_ret_t,vol_5
0,2018-03-20,63.310219,0.258843,0.683649,0.057507,6.0,0.201336,-0.040082,-0.042018,0.012110,...,0.056060,0.029845,1.0,-1.0,-0.006030,0.006030,0,0,NaN,NaN
1,2018-03-21,62.929611,0.079694,0.891288,0.029018,3.0,0.050676,-0.055223,-0.005217,0.019322,...,0.030065,0.011303,1.0,-1.0,-0.026154,0.026154,0,1,0.006030,NaN
2,2018-03-22,61.305096,0.348509,0.226298,0.425194,5.0,-0.076685,-0.029817,0.008005,0.017561,...,0.040454,0.003072,1.0,-1.0,-0.027012,0.027012,0,1,0.026154,NaN
3,2018-03-23,59.671288,0.241439,0.482122,0.276438,5.0,-0.034999,-0.015262,-0.058099,-0.005390,...,-0.006712,0.049573,1.0,1.0,0.037256,0.037256,0,1,0.027012,NaN
4,2018-03-26,61.936344,0.234637,0.625601,0.139762,6.0,0.094875,-0.029753,-0.008020,0.026985,...,0.043457,0.037637,1.0,-1.0,-0.032600,0.032600,0,1,0.037256,NaN
5,2018-03-27,59.949760,0.075072,0.531589,0.393340,7.0,-0.318268,-0.059012,-0.016827,0.007342,...,0.044529,0.055805,1.0,-1.0,-0.006680,0.006680,0,0,0.032600,0.025810
6,2018-03-28,59.550602,0.310788,0.477132,0.212080,9.0,0.098708,-0.038110,-0.029604,0.028321,...,-0.004168,0.057462,1.0,1.0,0.019604,0.019604,0,1,0.006680,0.025941
7,2018-03-29,60.729534,0.370141,0.613385,0.016474,3.0,0.353667,-0.021918,0.029772,0.047880,...,0.060755,0.026523,1.0,-1.0,-0.024134,0.024134,0,1,0.019604,0.024631
8,2018-04-02,59.281403,0.134719,0.351496,0.513785,10.0,-0.379066,-0.038686,-0.003821,0.047725,...,0.017328,0.019723,1.0,1.0,0.009817,0.009817,0,0,0.024134,0.024055
9,2018-04-03,59.866215,0.196441,0.353618,0.449942,9.0,-0.253501,-0.014222,-0.011880,0.029031,...,0.019184,0.024981,1.0,1.0,0.013859,0.013859,0,1,0.009817,0.018567


In [5]:
df.isna().sum()

Date                   0
Price                  0
avg_positive_XLK      12
avg_neutral_XLK       12
avg_negative_XLK      12
                      ..
AbsReturn_next_day     1
no_news                0
HighVol_next_day       0
abs_ret_t              1
vol_5                  5
Length: 399, dtype: int64

In [6]:
rows_with_nan = df[df.isna().any(axis=1)]
rows_with_nan


,Date,Price,avg_positive_XLK,avg_neutral_XLK,avg_negative_XLK,n_XLK,sent_index_XLK,emb_0,emb_1,emb_2,...,emb_382,emb_383,is_trading_day,Sign_next_day,Return_next_day,AbsReturn_next_day,no_news,HighVol_next_day,abs_ret_t,vol_5
0,2018-03-20,63.310219,0.258843,0.683649,0.057507,6.0,0.201336,-0.040082,-0.042018,0.012110,...,0.056060,0.029845,1.0,-1.0,-0.006030,0.006030,0,0,NaN,NaN
1,2018-03-21,62.929611,0.079694,0.891288,0.029018,3.0,0.050676,-0.055223,-0.005217,0.019322,...,0.030065,0.011303,1.0,-1.0,-0.026154,0.026154,0,1,0.006030,NaN
2,2018-03-22,61.305096,0.348509,0.226298,0.425194,5.0,-0.076685,-0.029817,0.008005,0.017561,...,0.040454,0.003072,1.0,-1.0,-0.027012,0.027012,0,1,0.026154,NaN
3,2018-03-23,59.671288,0.241439,0.482122,0.276438,5.0,-0.034999,-0.015262,-0.058099,-0.005390,...,-0.006712,0.049573,1.0,1.0,0.037256,0.037256,0,1,0.027012,NaN
4,2018-03-26,61.936344,0.234637,0.625601,0.139762,6.0,0.094875,-0.029753,-0.008020,0.026985,...,0.043457,0.037637,1.0,-1.0,-0.032600,0.032600,0,1,0.037256,NaN
15,2018-04-11,61.035892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,0.011944,0.011944,1,0,0.005914,0.013394
101,2018-08-13,67.940887,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,0.006152,0.006152,1,0,0.001370,0.003274
113,2018-08-29,70.586601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-1.0,-0.003040,0.003040,1,0,0.009015,0.006410
197,2018-12-31,58.187222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,0.000645,0.000645,1,0,0.009402,0.020910
243,2019-03-08,65.538071,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,0.021259,0.021259,1,1,0.000144,0.004336


In [7]:
df = df.dropna()
df.isna().sum()

Date                  0
Price                 0
avg_positive_XLK      0
avg_neutral_XLK       0
avg_negative_XLK      0
                     ..
AbsReturn_next_day    0
no_news               0
HighVol_next_day      0
abs_ret_t             0
vol_5                 0
Length: 399, dtype: int64

In [8]:
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from scipy.stats import randint, uniform

drop_cols = ["Date", "HighVol_next_day", "Return_next_day", "AbsReturn_next_day", "Return", "Sign"]
X = df.drop(columns=[c for c in drop_cols if c in df.columns])
y = df["HighVol_next_day"]

catboost = CatBoostClassifier(
    loss_function="Logloss",
    eval_metric="AUC",
    random_seed=12,
    verbose=0,
    allow_writing_files=False
)

param_distributions = {
    "depth": randint(3, 7),
    "learning_rate": uniform(0.01, 0.19),
    "iterations": randint(100, 500),
    "l2_leaf_reg": uniform(1, 14),
    "border_count": randint(32, 128),
    "random_strength": uniform(0.5, 5),
    "min_data_in_leaf": randint(5, 30),
    "rsm": uniform(0.6, 0.4),
    "boosting_type": ["Ordered", "Plain"],
    "bootstrap_type": ["Bernoulli"],# , "MVS"],
    "subsample": uniform(0.6, 0.4),
}

tscv = TimeSeriesSplit(n_splits=5)

random_search = RandomizedSearchCV(
    estimator=catboost,
    param_distributions=param_distributions,
    n_iter=40,
    cv=tscv,
    scoring="roc_auc",
    random_state=12,
    n_jobs=-1,
    verbose=2,
    return_train_score=True
)

random_search.fit(X, y)

print(f"Best AUC: {random_search.best_score_:.4f}")
print("Best parameters:")
for p, v in random_search.best_params_.items():
    print(f"  {p}: {v}")


Fitting 5 folds for each of 40 candidates, totalling 200 fits


c:\Users\leszc\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
1 fits failed out of a total of 200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\leszc\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\leszc\anaconda3\Lib\site-packages\catboost\core.py", line 5245, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, graph, sample_weight, None, None, None, None, baseline, use_best_model,
  File "c:\Users\leszc\anaconda3\Lib\site-packages\catboost\core.py", line 2

Best AUC: 0.5878
Best parameters:
  boosting_type: Plain
  bootstrap_type: Bernoulli
  border_count: 36
  depth: 4
  iterations: 384
  l2_leaf_reg: 9.446575612428736
  learning_rate: 0.02368087844483039
  min_data_in_leaf: 9
  random_strength: 2.529791714798214
  rsm: 0.9786223289451212
  subsample: 0.9428762235327774


In [9]:
q = 0.80  # try 0.70, 0.75, 0.80
tau = df["AbsReturn_next_day"].quantile(q)
df["HighVol_next_day"] = (df["AbsReturn_next_day"] > tau).astype(int)


df["abs_ret_t"] = np.log(df["Price"] / df["Price"].shift(1)).abs()
df["vol_5"] = df["abs_ret_t"].rolling(5).mean()


df = df.dropna(subset=["HighVol_next_day"])
df.head(10)

,Date,Price,avg_positive_XLK,avg_neutral_XLK,avg_negative_XLK,n_XLK,sent_index_XLK,emb_0,emb_1,emb_2,...,emb_382,emb_383,is_trading_day,Sign_next_day,Return_next_day,AbsReturn_next_day,no_news,HighVol_next_day,abs_ret_t,vol_5
5,2018-03-27,59.949760,0.075072,0.531589,0.393340,7.0,-0.318268,-0.059012,-0.016827,0.007342,...,0.044529,0.055805,1.0,-1.0,-0.006680,0.006680,0,0,NaN,NaN
6,2018-03-28,59.550602,0.310788,0.477132,0.212080,9.0,0.098708,-0.038110,-0.029604,0.028321,...,-0.004168,0.057462,1.0,1.0,0.019604,0.019604,0,1,0.006680,NaN
7,2018-03-29,60.729534,0.370141,0.613385,0.016474,3.0,0.353667,-0.021918,0.029772,0.047880,...,0.060755,0.026523,1.0,-1.0,-0.024134,0.024134,0,1,0.019604,NaN
8,2018-04-02,59.281403,0.134719,0.351496,0.513785,10.0,-0.379066,-0.038686,-0.003821,0.047725,...,0.017328,0.019723,1.0,1.0,0.009817,0.009817,0,0,0.024134,NaN
9,2018-04-03,59.866215,0.196441,0.353618,0.449942,9.0,-0.253501,-0.014222,-0.011880,0.029031,...,0.019184,0.024981,1.0,1.0,0.013859,0.013859,0,0,0.009817,NaN
10,2018-04-04,60.701687,0.125011,0.321051,0.553937,6.0,-0.428926,-0.037838,-0.008110,0.063454,...,0.018787,0.020253,1.0,1.0,0.004121,0.004121,0,0,0.013859,0.014819
11,2018-04-05,60.952347,0.100165,0.886647,0.013189,1.0,0.086976,0.108407,-0.035265,0.074456,...,0.005906,-0.028267,1.0,-1.0,-0.024826,0.024826,0,1,0.004121,0.014307
12,2018-04-06,59.457771,0.578852,0.035447,0.385701,1.0,0.193151,-0.079174,0.013276,0.026213,...,-0.053769,0.055850,1.0,1.0,0.007776,0.007776,0,0,0.024826,0.015351
13,2018-04-09,59.921917,0.133793,0.492447,0.373760,5.0,-0.239967,-0.035168,-0.004428,0.025551,...,-0.009289,0.030137,1.0,1.0,0.024334,0.024334,0,1,0.007776,0.012080
14,2018-04-10,61.397930,0.028401,0.226109,0.745490,3.0,-0.717089,0.006485,-0.028084,-0.005789,...,0.039189,-0.004194,1.0,-1.0,-0.005914,0.005914,0,0,0.024334,0.014983


In [10]:
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from scipy.stats import randint, uniform

drop_cols = ["Date", "HighVol_next_day", "Return_next_day", "AbsReturn_next_day", "Return", "Sign"]
X = df.drop(columns=[c for c in drop_cols if c in df.columns])
y = df["HighVol_next_day"]

catboost = CatBoostClassifier(
    loss_function="Logloss",
    eval_metric="AUC",
    random_seed=12,
    verbose=0,
    allow_writing_files=False
)

param_distributions = {
    "depth": randint(3, 7),
    "learning_rate": uniform(0.01, 0.19),
    "iterations": randint(100, 500),
    "l2_leaf_reg": uniform(1, 14),
    "border_count": randint(32, 128),
    "random_strength": uniform(0.5, 5),
    "min_data_in_leaf": randint(5, 30),
    "rsm": uniform(0.6, 0.4),
    "boosting_type": ["Ordered", "Plain"],
    "bootstrap_type": ["Bernoulli"],# , "MVS"],
    "subsample": uniform(0.6, 0.4),
}

tscv = TimeSeriesSplit(n_splits=5)

random_search = RandomizedSearchCV(
    estimator=catboost,
    param_distributions=param_distributions,
    n_iter=40,
    cv=tscv,
    scoring="roc_auc",
    random_state=12,
    n_jobs=-1,
    verbose=2,
    return_train_score=True
)

random_search.fit(X, y)

print(f"Best AUC: {random_search.best_score_:.4f}")
print("Best parameters:")
for p, v in random_search.best_params_.items():
    print(f"  {p}: {v}")


Fitting 5 folds for each of 40 candidates, totalling 200 fits
Best AUC: 0.5926
Best parameters:
  boosting_type: Plain
  bootstrap_type: Bernoulli
  border_count: 96
  depth: 5
  iterations: 428
  l2_leaf_reg: 12.147580642880225
  learning_rate: 0.06531042413892015
  min_data_in_leaf: 16
  random_strength: 1.140965512530431
  rsm: 0.819061001176923
  subsample: 0.7166093786623786


In [11]:
q = 0.75  # try 0.70, 0.75, 0.80
tau = df["AbsReturn_next_day"].quantile(q)
df["HighVol_next_day"] = (df["AbsReturn_next_day"] > tau).astype(int)


df["abs_ret_t"] = np.log(df["Price"] / df["Price"].shift(1)).abs()
df["vol_5"] = df["abs_ret_t"].rolling(5).mean()


df = df.dropna(subset=["HighVol_next_day"])
df.head(10)

,Date,Price,avg_positive_XLK,avg_neutral_XLK,avg_negative_XLK,n_XLK,sent_index_XLK,emb_0,emb_1,emb_2,...,emb_382,emb_383,is_trading_day,Sign_next_day,Return_next_day,AbsReturn_next_day,no_news,HighVol_next_day,abs_ret_t,vol_5
5,2018-03-27,59.949760,0.075072,0.531589,0.393340,7.0,-0.318268,-0.059012,-0.016827,0.007342,...,0.044529,0.055805,1.0,-1.0,-0.006680,0.006680,0,0,NaN,NaN
6,2018-03-28,59.550602,0.310788,0.477132,0.212080,9.0,0.098708,-0.038110,-0.029604,0.028321,...,-0.004168,0.057462,1.0,1.0,0.019604,0.019604,0,1,0.006680,NaN
7,2018-03-29,60.729534,0.370141,0.613385,0.016474,3.0,0.353667,-0.021918,0.029772,0.047880,...,0.060755,0.026523,1.0,-1.0,-0.024134,0.024134,0,1,0.019604,NaN
8,2018-04-02,59.281403,0.134719,0.351496,0.513785,10.0,-0.379066,-0.038686,-0.003821,0.047725,...,0.017328,0.019723,1.0,1.0,0.009817,0.009817,0,0,0.024134,NaN
9,2018-04-03,59.866215,0.196441,0.353618,0.449942,9.0,-0.253501,-0.014222,-0.011880,0.029031,...,0.019184,0.024981,1.0,1.0,0.013859,0.013859,0,0,0.009817,NaN
10,2018-04-04,60.701687,0.125011,0.321051,0.553937,6.0,-0.428926,-0.037838,-0.008110,0.063454,...,0.018787,0.020253,1.0,1.0,0.004121,0.004121,0,0,0.013859,0.014819
11,2018-04-05,60.952347,0.100165,0.886647,0.013189,1.0,0.086976,0.108407,-0.035265,0.074456,...,0.005906,-0.028267,1.0,-1.0,-0.024826,0.024826,0,1,0.004121,0.014307
12,2018-04-06,59.457771,0.578852,0.035447,0.385701,1.0,0.193151,-0.079174,0.013276,0.026213,...,-0.053769,0.055850,1.0,1.0,0.007776,0.007776,0,0,0.024826,0.015351
13,2018-04-09,59.921917,0.133793,0.492447,0.373760,5.0,-0.239967,-0.035168,-0.004428,0.025551,...,-0.009289,0.030137,1.0,1.0,0.024334,0.024334,0,1,0.007776,0.012080
14,2018-04-10,61.397930,0.028401,0.226109,0.745490,3.0,-0.717089,0.006485,-0.028084,-0.005789,...,0.039189,-0.004194,1.0,-1.0,-0.005914,0.005914,0,0,0.024334,0.014983


In [13]:
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from scipy.stats import randint, uniform

drop_cols = ["Date", "HighVol_next_day", "Return_next_day", "AbsReturn_next_day", "Return", "Sign"]
X = df.drop(columns=[c for c in drop_cols if c in df.columns])
y = df["HighVol_next_day"]

catboost = CatBoostClassifier(
    loss_function="Logloss",
    eval_metric="AUC",
    random_seed=12,
    verbose=0,
    allow_writing_files=False
)

param_distributions = {
    "depth": randint(3, 7),
    "learning_rate": uniform(0.01, 0.19),
    "iterations": randint(100, 500),
    "l2_leaf_reg": uniform(1, 14),
    "border_count": randint(32, 128),
    "random_strength": uniform(0.5, 5),
    "min_data_in_leaf": randint(5, 30),
    "rsm": uniform(0.6, 0.4),
    "boosting_type": ["Plain"],
    "bootstrap_type": ["Bernoulli"],# , "MVS"],
    "subsample": uniform(0.6, 0.4),
}

tscv = TimeSeriesSplit(n_splits=5)

random_search = RandomizedSearchCV(
    estimator=catboost,
    param_distributions=param_distributions,
    n_iter=40,
    cv=tscv,
    scoring="roc_auc",
    random_state=12,
    n_jobs=-1,
    verbose=2,
    return_train_score=True
)

random_search.fit(X, y)

print(f"Best AUC: {random_search.best_score_:.4f}")
print("Best parameters:")
for p, v in random_search.best_params_.items():
    print(f"  {p}: {v}")


Fitting 5 folds for each of 40 candidates, totalling 200 fits
Best AUC: 0.5910
Best parameters:
  boosting_type: Plain
  bootstrap_type: Bernoulli
  border_count: 101
  depth: 4
  iterations: 193
  l2_leaf_reg: 6.0923294629361155
  learning_rate: 0.07337662082108412
  min_data_in_leaf: 20
  random_strength: 3.221779390653432
  rsm: 0.6815004245020413
  subsample: 0.9368931088227698
